<a href="https://colab.research.google.com/github/sandeep92134/The-Data-Visualization-Workshop-by-packt/blob/master/module%206/Exercise6.04%3A%20Plotting%20Stock%20Price%20Data%20in%20Tabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 6.04: Plotting Stock Price Data in Tabs

We will revisit the basic widgets and build a simple plot that will display the first 25 data points for the selected stock. The displayed stock can be changed with a drop-down menu.

The dataset of this exercise contains temporal stock price data.   
This means we'll be looking at data over a range of time.   
> The dataset for this exercise is quite big and has to be downloaded and added into the data folder.   
The dataset is available in the data folder of the GitHub repository; here is the link to it: https://bit.ly/2UaLtSV

#### Loading our dataset

In [1]:
# importing the necessary dependencies
import pandas as pd

In [2]:
# make bokeh display figures inside the notebook
from bokeh.io import output_notebook

output_notebook()

In [3]:
# loading the Dataset with geoplotlib
dataset = pd.read_csv('https://raw.githubusercontent.com/sandeep92134/The-Data-Visualization-Workshop-by-packt/master/module%206/datasets/stock_prices.csv')

In [4]:
# looking at the dataset
dataset.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


Since the date column has no information about the hour, minute, and second, we want to avoid displaying them in the visualization later on and simply display the year, month, and day.   

Therefore we'll create a new column that holds the formatted short version of the date value. 

In [5]:
# mapping the date of each row to only the year-month-day format
from datetime import datetime

def shorten_time_stamp(timestamp):
    shortened = timestamp[0]
    
    if len(shortened) > 10:
        parsed_date=datetime.strptime(shortened, '%Y-%m-%d %H:%M:%S')
        shortened=datetime.strftime(parsed_date, '%Y-%m-%d')
    
    return shortened

dataset['short_date'] = dataset.apply(lambda x: shorten_time_stamp(x), axis=1)

dataset.head()

,date,symbol,open,close,low,high,volume,short_date
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0,2016-01-05
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0,2016-01-06
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0,2016-01-07
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0,2016-01-08
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0,2016-01-11


**Note:**   
The exectuion of the cell will take a moment since it's a fairly large dataset.   
Please be patient.   

The last, newly added, column now holds the timestamp without the hour, minute, and second information.  

---

#### Creating a basic plot and adding a widget

In this task, we will create a basic visualization with the stock price dataset. This will be your first interactive visualization in which you can dynamically change the stock that is displayed in the graph.   

We will get used to one of the above described interactive widgets, the dropdown menu. It will be the main point of interaction for our visualization.

In [6]:
# importing the necessary dependencies 
from ipywidgets import interact

from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import figure, show

In order to structure our notebook better, we want to write a adaptable method that gets a subsection of stock data as argument and builds a two tab `Pane` object, that lets us switch between the two views in our visualization.   
The first tab will contain a line plot of the given data, the second one a circle-based representation of the same data.   
A legend will display the name of the currently viewed at stock.

In [7]:
# method to build the tab-based plot
def get_plot(stock):
    stock_name=stock['symbol'].unique()[0]

    line_plot=figure(title='Stock prices', 
                     x_axis_label='Date', x_range=stock['short_date'], 
                     y_axis_label='Price in $USD')
    line_plot.line(stock['short_date'], stock['high'], legend_label=stock_name)
    line_plot.xaxis.major_label_orientation = 1

    circle_plot=figure(title='Stock prices', 
                       x_axis_label='Date', x_range=stock['short_date'], 
                       y_axis_label='Price in $USD')
    circle_plot.circle(stock['short_date'], stock['high'], legend_label=stock_name)
    circle_plot.xaxis.major_label_orientation = 1
    
    line_tab=Panel(child=line_plot, title='Line')
    circle_tab=Panel(child=circle_plot, title='Circles')
    tabs = Tabs(tabs=[ line_tab, circle_tab ])

    return tabs


Before we can build our interaction, we have to get a list of all the stock names that are present in the dataset.   
Once we have done that, we can then use this list as an input for the interact element.   
With each interaction of the dropdown, our displayed data will then be updated.   

To keep it simple, we only want to display the first 25 entries of each stock in this task.   
By default, the stock of Apple should be displayed, it's symbol in the dataset is 'AAPL'.

In [8]:
# extracing all the stock names
stock_names=dataset['symbol'].unique()

In [9]:
# creating the dropdown interaction and building the plot
# based on selection

@interact(Stock=stock_names)
def get_stock_for(Stock='AAPL'):
    stock = dataset[dataset['symbol'] == Stock][:25]
    show(get_plot(stock))

interactive(children=(Dropdown(description='Stock', index=4, options=('WLTW', 'A', 'AAL', 'AAP', 'AAPL', 'ABC'…

**Note:**   
We can already see that each date is displayed on the x-axis. If we want to display a bigger time range, we have to customize the ticks on our x-axis. This can be done using `Ticker` objects.   